<a href="https://colab.research.google.com/github/GeorgeThan414/Machine_Learning_CSD/blob/main/Exercise_9_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install libraries

# Task 1


In [ ]:
!pip install swig

In [ ]:
!pip install "gymnasium[box2d]"

In [ ]:
!pip install renderlab

# Task 2 & 3
Describe the environment with below techniques:
1) Observation Space
2) Action Space
3) Reward Function

#Observation space
Observation space consist of continuing variable which consist of 8 distinct values in LunarLander-v3. These values are the following:  

1) Horizontal position
2) Vertical position
3) Horizontal velocity
4) Vertical velocity
5) Spacecraft angle
6) Angle velocity
7) Left side contact with ground
8) Right side contact with ground

# Action Space

Action space consist of discrete variables in this problem. These variables are the following:

1) No push
2) Activation of left engine
3) Activation of main engine
4) Activation of irght engine

#Reward Function

This function gives the agent his rewards, based on the rewards agent decides which will be the next movement, of course following the optimal policy or the predefined policy that dev assigned to the agent.

Whether the rewards are positive, that means that the agent is going to achieve his goal, to land the spacecraft inside the two flags.

On the other head, negative rewards shows that the agent is not performing well, the following policy may be not the optimal, and his decisions is not going to achieve his goal.

# Initialization of a Random Agent - Task 4


Random Agent cannot be learnt any information from episode to episode, his actions are randomly selected. It can be used for visualization purposes.

In [ ]:
import gymnasium as gym
import renderlab as r1
import time
# Initialize the environment
env = gym.make("LunarLander-v3", render_mode="rgb_array")
env= r1.RenderFrame(env, "./output")

obs, info = env.reset()

while True:
  # Make an action
  action = env.action_space.sample()

  # From the above action get the obs, rewards, if the episode is terminated or truncated and general info.
  # Also moves to the next step is the episode is still activated.
  obs, reward, terminated, truncated, info = env.step(action)

  if terminated or truncated:
    break

  time.sleep(0.05)

env.play()

Moviepy - Building video temp-{start}.mp4.
Moviepy - Writing video temp-{start}.mp4



Moviepy - Done !
Moviepy - video ready temp-{start}.mp4


# Task 5

In [32]:
import numpy as np
env = gym.make("LunarLander-v3", render_mode="rgb_array")
env = r1.RenderFrame(env, "./output")

num_episodes = 5
episode_scores = []

for episode in range(num_episodes):
    obs, info = env.reset()
    total_reward = 0

    while True:
        action = env.action_space.sample()  # random agent
        obs, reward, terminated, truncated, info = env.step(action)
        total_reward += reward

        if terminated or truncated:
          break

    time.sleep(0.05)

    episode_scores.append(total_reward)
    print(f"Episode {episode+1} score: {total_reward:.2f}")

# Play recorded episodes
env.play()

mean_score = np.mean(episode_scores)
print(f"\nMean score over {num_episodes} episodes: {mean_score:.2f}")

Episode 1 score: -86.59
Episode 2 score: -312.71
Episode 3 score: -220.22
Episode 4 score: -283.14
Episode 5 score: -147.38
Moviepy - Building video temp-{start}.mp4.
Moviepy - Writing video temp-{start}.mp4



Moviepy - Done !
Moviepy - video ready temp-{start}.mp4



Mean score over 5 episodes: -210.01


#Task 6 - PPO & DQN

# Install the baseline library

In [33]:
!pip install stable-baselines3[extra]

In [42]:
from stable_baselines3 import DQN

env = gym.make("LunarLander-v3")
model_dqn = DQN(
    policy="MlpPolicy",
    env=env,
    learning_rate=1e-3,
    buffer_size=100_000,
    learning_starts=1_000,
    batch_size=64,
    gamma=0.99,
    target_update_interval=1000,
    verbose=0)

model_dqn.learn(total_timesteps=200_000)
model_dqn.save("dqn_lunarlander")
env.close()


In [44]:
from stable_baselines3 import PPO

env = gym.make("LunarLander-v3")
model_ppo = PPO(
    policy="MlpPolicy",
    env=env,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.2,
    verbose=0)

model_ppo.learn(total_timesteps=200_000)
model_ppo.save("ppo_lunarlander")

env.close()

# Evaluate the model - Task 7

In [59]:

def evaluate_agent(model=None, env_name="LunarLander-v3", n_episodes=5):
    env = gym.make(env_name)
    scores = []

    for _ in range(n_episodes):
        obs, info = env.reset()
        done = False
        total_reward = 0

        while not done:
            if model is None:
                action = env.action_space.sample()   # random agent
            else:
                action, _ = model.predict(obs, deterministic=True)

            obs, reward, terminated, truncated, info = env.step(action)
            total_reward += reward
            done = terminated or truncated

        scores.append(total_reward)

    env.close()
    return np.mean(scores), scores

In [53]:
# Random agent
mean_random, _ = evaluate_agent()

# DQN
model_dqn = DQN.load("dqn_lunarlander")
mean_dqn, _ = evaluate_agent(model_dqn)

# PPO
model_ppo = PPO.load("ppo_lunarlander")
mean_ppo, _ = evaluate_agent(model_ppo)

print("Mean rewards (5 episodes)")
print(f"Random Agent: {mean_random:.2f}")
print(f"DQN:          {mean_dqn:.2f}")
print(f"PPO:          {mean_ppo:.2f}")

Mean rewards (5 episodes)
Random Agent: -133.27
DQN:          -17.77
PPO:          68.22


# Task 8

The state-of-the art implementation agents playing LunarLander-v2 are the following:

#DQN Algorithm
https://huggingface.co/sb3/dqn-LunarLander-v2

#PPO Algorithm
https://huggingface.co/sb3/ppo-LunarLander-v2

# DQN - Parameters HuggingFace

In [54]:
env = gym.make("LunarLander-v3")

model_dqn = DQN(
    policy="MlpPolicy",
    env=env,
    learning_rate=6.3e-4,
    buffer_size=50_000,
    batch_size=128,
    gamma=0.99,
    train_freq=4,
    target_update_interval=250,
    exploration_fraction=0.12,
    exploration_final_eps=0.10,
    learning_starts=0,
    policy_kwargs=dict(net_arch=[256, 256]),
    verbose=0
)

model_dqn.learn(total_timesteps=10_000)
model_dqn.save("dqn_lunarlander_v3_10k")
env.close()

# PPO - Parameters HuggingFace

The only parameter that the documentation doesnt provide is the learning rate, so the learning rate will be the default value from SB3.

In [55]:
from stable_baselines3.common.env_util import make_vec_env
vec_env = make_vec_env("LunarLander-v3", n_envs=16)

model_ppo = PPO(
    policy="MlpPolicy",
    env=vec_env,
    n_steps=1024,
    batch_size=64,
    n_epochs=4,
    gamma=0.999,
    gae_lambda=0.98,
    ent_coef=0.01,
    learning_rate=3e-4,   # SB3 default
    verbose=0
)

model_ppo.learn(total_timesteps=10_000)
model_ppo.save("ppo_lunarlander_v3_10k")
vec_env.close()

In [56]:
# DQN
model_dqn = DQN.load("dqn_lunarlander_v3_10k")
mean_dqn, _ = evaluate_agent(model_dqn)

# PPO
model_ppo = PPO.load("ppo_lunarlander_v3_10k")
mean_ppo, _ = evaluate_agent(model_ppo)

print("Mean rewards (5 episodes)")
print(f"Optimized DQN:          {mean_dqn:.2f}")
print(f"Optimized PPO:          {mean_ppo:.2f}")

Mean rewards (5 episodes)
Optimized DQN:          -48.30
Optimized PPO:          -121.58
